#ML with Pyspark

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 62.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=bbd12c303f0cf531a5aca3592ea0abef7ab42908460654817b7c2733ad63ab65
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
#Starting the session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ML_Concepts").getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.csv("/content/drive/MyDrive/Pyspark/test1.csv", header =True, inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
#We are putting a independent variable in a list
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols= ["age", "Experience"], outputCol="Independent_variable")

In [12]:
output = feature_assembler.transform(df)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent_variable|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
finalised_data = output.select(["Independent_variable", "salary"])

In [15]:
finalised_data.show()

+--------------------+------+
|Independent_variable|salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
#Regression Ml
from pyspark.ml.regression import LinearRegression

#Splitting the data into train test split
train_data, test_data = finalised_data.randomSplit([0.75, 0.25])

regressor = LinearRegression(featuresCol = "Independent_variable", labelCol="salary")
regressor = regressor.fit(train_data)

In [20]:
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [21]:
regressor.intercept

19919.060052215827

In [25]:
pred = regressor.evaluate(test_data)

In [28]:
pred.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------+----------------+
|Independent_variable|salary|      prediction|
+--------------------+------+----------------+
|          [29.0,4.0]| 20000|19342.0365535255|
+--------------------+------+----------------+



In [29]:
pred.meanAbsoluteError, pred.meanSquaredError

(657.9634464744995, 432915.8968966016)